In [4]:
import serial
import time

# Configure COM port
PORT = "COM4"       # Replace with your port
BAUD = 10000000       # Must match STM32's baud rate

try:
    ser = serial.Serial(PORT, BAUD, timeout=1)
    print(f"Connected to {PORT} at {BAUD} baud.\n")
    time.sleep(2)  # Allow connection to stabilize

    while True:
        if ser.in_waiting > 0:
            message = ser.readline().decode('utf-8', errors='ignore').strip()
            if message:
                print(f"Received: {message}")

except serial.SerialException as e:
    print(f"Error: {e}")
except KeyboardInterrupt:
    print("\nStopped by user.")
finally:
    if 'ser' in locals() and ser.is_open:
        ser.close()
        print("Serial port closed.")


Connected to COM4 at 10000000 baud.

Received: Hello via LPUART1

Stopped by user.
Serial port closed.


In [5]:
import serial
import time

PORT = "COM4"
BAUD = 10000000   # 10 Mbps

try:
    ser = serial.Serial(PORT, BAUD, timeout=1)
    print(f"Connected to {PORT} at {BAUD} baud.\n")
    time.sleep(2)  # Allow connection to stabilize

    while True:
        if ser.in_waiting > 0:
            # Read one line
            message = ser.readline().decode('utf-8', errors='ignore').strip()
            if message:
                print(f"Received: {message}")

                # Echo it back to STM32
                ser.write((message + "\r\n").encode('utf-8'))
                ser.flush()

except serial.SerialException as e:
    print(f"Error: {e}")
except KeyboardInterrupt:
    print("\nStopped by user.")
finally:
    if 'ser' in locals() and ser.is_open:
        ser.close()
        print("Serial port closed.")


Connected to COM4 at 10000000 baud.

Received: Hello via LPUART1
Received: Hello via LPUART1

Stopped by user.
Serial port closed.


In [6]:
import serial
import time

PORT = "COM4"
BAUD = 10000000   # 10 Mbps

try:
    ser = serial.Serial(PORT, BAUD, timeout=2)
    print(f"Connected to {PORT} at {BAUD} baud.\n")
    time.sleep(2)  # Allow connection to stabilize

    msg = "Hello via LPUART1\r\n"

    # 1. Send message once
    ser.write(msg.encode('utf-8'))
    ser.flush()
    print(f"Sent: {msg.strip()}")

    # 2. Read echoed message once
    echoed = ser.readline().decode('utf-8', errors='ignore').strip()

    if echoed:
        print(f"Received: {echoed}")
    else:
        print("No response received (timeout).")

except serial.SerialException as e:
    print(f"Error: {e}")
finally:
    if 'ser' in locals() and ser.is_open:
        ser.close()
        print("Serial port closed.")


Connected to COM4 at 10000000 baud.

Sent: Hello via LPUART1
Received: Hello via LPUART1
Serial port closed.


In [7]:
import serial
import time

PORT = "COM4"
BAUD = 10000000   # 10 Mbps

try:
    ser = serial.Serial(PORT, BAUD, timeout=2)
    print(f"Connected to {PORT} at {BAUD} baud.\n")
    time.sleep(2)  # Allow connection to stabilize

    msg = "Hello via LPUART1\r\n"

    # 1. Start timer and send message once
    start_time = time.time()
    ser.write(msg.encode('utf-8'))
    ser.flush()
    print(f"Sent: {msg.strip()}")

    # 2. Read echoed message once
    echoed = ser.readline().decode('utf-8', errors='ignore').strip()
    end_time = time.time()

    if echoed:
        elapsed_ms = (end_time - start_time) * 1000
        print(f"Received: {echoed}")
        print(f"Round-trip time: {elapsed_ms:.2f} ms")
    else:
        print("No response received (timeout).")

except serial.SerialException as e:
    print(f"Error: {e}")
finally:
    if 'ser' in locals() and ser.is_open:
        ser.close()
        print("Serial port closed.")


Connected to COM4 at 10000000 baud.

Sent: Hello via LPUART1
Received: Hello via LPUART1
Round-trip time: 0.00 ms
Serial port closed.


In [1]:
import serial
import time
import os

PORT = "COM4"
BAUD = 12000000     
BUF_SIZE = 40000    

try:
    ser = serial.Serial(PORT, BAUD, timeout=5)  
    print(f"Connected to {PORT} at {BAUD} baud.\n")
    time.sleep(2) 

    msg = os.urandom(BUF_SIZE)

    start_time = time.time()
    ser.write(msg)
    ser.flush()
    print(f"Sent {BUF_SIZE} bytes.")

    echoed = ser.read(BUF_SIZE)
    end_time = time.time()

    if len(echoed) == BUF_SIZE:
        elapsed_ms = (end_time - start_time) * 1000
        print(f"Received {len(echoed)} bytes successfully.")
        print(f"Round-trip time: {elapsed_ms:.2f} ms")

        if echoed == msg:
            print("Data integrity check: Passed")
        else:
            print("Data integrity check: Failed")
    else:
        print(f"Timeout or incomplete data received ({len(echoed)} bytes).")

except serial.SerialException as e:
    print(f"Error: {e}")
finally:
    if 'ser' in locals() and ser.is_open:
        ser.close()
        print("Serial port closed.")


Connected to COM4 at 12000000 baud.

Sent 40000 bytes.
Received 40000 bytes successfully.
Round-trip time: 70.43 ms
Data integrity check: Passed
Serial port closed.


In [31]:
import serial
import time
import os
from datetime import datetime
import struct  # add at top

PORT = "COM4"
BAUD = 12000000         # or 12000000 if STM32 is set that way
BUF_SIZE = 40000      # size of random test buffer

def read_exact(ser, n, timeout_s=5):
    """Read exactly n bytes or return what we got by timeout."""
    end = time.time() + timeout_s
    buf = bytearray()
    while len(buf) < n and time.time() < end:
        chunk = ser.read(n - len(buf))
        if chunk:
            buf += chunk
        else:
            time.sleep(0.001)
    return bytes(buf)

def timestamp_ms():
    """Return current time string with millisecond resolution."""
    return datetime.now().strftime("%H:%M:%S.%f")[:-3]

try:
    ser = serial.Serial(PORT, BAUD, timeout=0.1)
    print(f"Connected to {PORT} at {BAUD} baud.\n")
    time.sleep(2)

    # ---- One-time loopback test ----
    msg = os.urandom(BUF_SIZE)
    t0 = time.time()
    ser.write(msg)
    ser.flush()
    echoed = read_exact(ser, BUF_SIZE, timeout_s=5)
    t1 = time.time()

    if len(echoed) == BUF_SIZE:
        rtt_ms = (t1 - t0) * 1000
        print(f"[Loopback] Received {len(echoed)} bytes, RTT {rtt_ms:.2f} ms")
        print(f"[Loopback] Integrity: {'Passed' if echoed == msg else 'FAILED'}")
    else:
        print(f"[Loopback] Incomplete ({len(echoed)}/{BUF_SIZE})")

    # Clear any leftover bytes before RX-only
    ser.reset_input_buffer()

    # # ---- RX-only mode ----
    # print("\nEntering RX-only mode. Press Ctrl-C to stop.\n")
    # while True:
    #     line = ser.readline()   # read until '\n' or timeout
    #     if line:
    #         try:
    #             # text = line.decode("utf-8").strip()
    #             text = line
    #         except UnicodeDecodeError:
    #             text = line.hex(" ")
    #         print(f"[{timestamp_ms()}] {text}")

    # ---- RX-only mode ----
    # print("\nEntering RX-only mode. Press Ctrl-C to stop.\n")
    # buf = bytearray()

    # while True:
    #     chunk = ser.read(1024)  # read whatever is available
    #     if chunk:
    #         buf += chunk
    #         # process complete 4-byte words
    #         while len(buf) >= 4:
    #             value_bytes = buf[:4]
    #             del buf[:4]
    #             # STM32 is little-endian
    #             value = struct.unpack("<I", value_bytes)[0]
    #             print(f"[{timestamp_ms()}] {value}")

    # ---- RX-only mode ----
    # print("\nEntering RX-only mode. Press Ctrl-C to stop.\n")
    # buf = bytearray()

    # while True:
    #     chunk = ser.read(1024)  # read whatever is available
    #     if chunk:
    #         buf += chunk
    #         value_bytes = buf[:4]
    #         del buf[:4]
            
    #         # STM32 is little-endian
    #         value = struct.unpack("<I", value_bytes)[0]
    #         print(f"[{timestamp_ms()}] {value}")

    while True:
        data = ser.read(1)      # read a single byte
        if data:
            value = data[0]     # convert single byte to int
            print(f"[{timestamp_ms()}] {value}")
            ser.reset_input_buffer()  # flush any leftover bytes immediately


except serial.SerialException as e:
    print(f"Serial error: {e}")
except KeyboardInterrupt:
    print("\nStopped by user.")
finally:
    if 'ser' in locals() and ser.is_open:
        ser.close()
        print("Serial port closed.")


Connected to COM4 at 12000000 baud.

[Loopback] Received 40000 bytes, RTT 70.24 ms
[Loopback] Integrity: Passed
[17:06:27.088] 1
[17:06:27.187] 2
[17:06:27.288] 3
[17:06:27.388] 4
[17:06:27.488] 5
[17:06:27.588] 6
[17:06:27.687] 7
[17:06:27.788] 8
[17:06:27.888] 9
[17:06:27.988] 10
[17:06:28.088] 11
[17:06:28.188] 12
[17:06:28.288] 13
[17:06:28.388] 14
[17:06:28.488] 15
[17:06:28.588] 16
[17:06:28.688] 17
[17:06:28.788] 18
[17:06:28.888] 19
[17:06:28.988] 20
[17:06:29.088] 21
[17:06:29.187] 22
[17:06:29.288] 23
[17:06:29.388] 24
[17:06:29.488] 25
[17:06:29.588] 26
[17:06:29.687] 27
[17:06:29.788] 28
[17:06:29.888] 29
[17:06:29.988] 30
[17:06:30.087] 31
[17:06:30.188] 0
[17:06:30.288] 1
[17:06:30.388] 2
[17:06:30.488] 3
[17:06:30.587] 4
[17:06:30.688] 5
[17:06:30.788] 6
[17:06:30.888] 7
[17:06:30.987] 8
[17:06:31.088] 9
[17:06:31.188] 10
[17:06:31.288] 11
[17:06:31.388] 12
[17:06:31.487] 13
[17:06:31.588] 14
[17:06:31.688] 15
[17:06:31.788] 16
[17:06:31.888] 17
[17:06:31.988] 18
[17:06:

In [8]:
import serial
import time
import os
from datetime import datetime
import struct  # add at top

PORT = "COM4"
BAUD = 12000000         # or 12000000 if STM32 is set that way
BUF_SIZE = 40000      # size of random test buffer

def read_exact(ser, n, timeout_s=5):
    """Read exactly n bytes or return what we got by timeout."""
    end = time.time() + timeout_s
    buf = bytearray()
    while len(buf) < n and time.time() < end:
        chunk = ser.read(n - len(buf))
        if chunk:
            buf += chunk
        else:
            time.sleep(0.001)
    return bytes(buf)

def timestamp_ms():
    """Return current time string with millisecond resolution."""
    return datetime.now().strftime("%H:%M:%S.%f")[:-3]

try:
    ser = serial.Serial(PORT, BAUD, timeout=0.1)
    print(f"Connected to {PORT} at {BAUD} baud.\n")
    time.sleep(2)

    # ---- One-time loopback test ----
    msg = os.urandom(BUF_SIZE)
    t0 = time.time()
    ser.write(msg)
    ser.flush()
    echoed = read_exact(ser, BUF_SIZE, timeout_s=5)
    t1 = time.time()

    if len(echoed) == BUF_SIZE:
        rtt_ms = (t1 - t0) * 1000
        print(f"[Loopback] Received {len(echoed)} bytes, RTT {rtt_ms:.2f} ms")
        print(f"[Loopback] Integrity: {'Passed' if echoed == msg else 'FAILED'}")
    else:
        print(f"[Loopback] Incomplete ({len(echoed)}/{BUF_SIZE})")

    # Clear any leftover bytes before RX-only
    ser.reset_input_buffer()

    NUM_WORDS = 5000
    BYTES_PER_WORD = 4
    TOTAL_BYTES = NUM_WORDS * BYTES_PER_WORD

    WORDS_PER_CHUNK, N_CHUNKS = 5000, 4
    RX_BYTES = WORDS_PER_CHUNK * 4   # 20 kB expected each receive

    # Build 20,000-word buffer [0..19999]
    all_words = list(range(WORDS_PER_CHUNK * N_CHUNKS))
    tx_buf = struct.pack(f"<{len(all_words)}I", *all_words)

    idx = 2
    while True:
        buf = bytearray()
        while len(buf) < TOTAL_BYTES:
            chunk = ser.read(TOTAL_BYTES - len(buf))  # read the remainder
            if not chunk:
                continue  # wait for more data
            buf.extend(chunk)

        # Decode as 10 little-endian uint32_t values
        values = struct.unpack(f"<{NUM_WORDS}I", buf)
        print(f"[{timestamp_ms()}] {values[0]}")

        time.sleep(0.01)
        start = idx * RX_BYTES
        # ser.write(tx_buf[start:start+RX_BYTES]); ser.flush()
        idx = (idx + 1) % N_CHUNKS
        print(f"[{timestamp_ms()}] {idx}-th segment sent")



    # while True:
    #     data = ser.read(1)      # read a single byte
    #     if data:
    #         value = data[0]     # convert single byte to int
    #         print(f"[{timestamp_ms()}] {value}")
    #         ser.reset_input_buffer()  # flush any leftover bytes immediately


except serial.SerialException as e:
    print(f"Serial error: {e}")
except KeyboardInterrupt:
    print("\nStopped by user.")
finally:
    if 'ser' in locals() and ser.is_open:
        ser.close()
        print("Serial port closed.")


Connected to COM4 at 12000000 baud.

[Loopback] Incomplete (0/40000)

Stopped by user.
Serial port closed.


In [12]:
import serial
import time
import os
from datetime import datetime
import struct  # add at top

PORT = "COM4"
BAUD = 12000000         # or 12000000 if STM32 is set that way
BUF_SIZE = 40000      # size of random test buffer

def read_exact(ser, n, timeout_s=5):
    """Read exactly n bytes or return what we got by timeout."""
    end = time.time() + timeout_s
    buf = bytearray()
    while len(buf) < n and time.time() < end:
        chunk = ser.read(n - len(buf))
        if chunk:
            buf += chunk
        else:
            time.sleep(0.001)
    return bytes(buf)

def timestamp_ms():
    """Return current time string with millisecond resolution."""
    return datetime.now().strftime("%H:%M:%S.%f")[:-3]

try:
    ser = serial.Serial(PORT, BAUD, timeout=0.1)
    print(f"Connected to {PORT} at {BAUD} baud.\n")
    time.sleep(2)

    ser.reset_input_buffer()
    ser.flush()

    # ---- One-time loopback test ----
    msg = os.urandom(BUF_SIZE)
    msg2 = os.urandom(int(BUF_SIZE / 2))
    msg3 = os.urandom(int(BUF_SIZE / 20))
    t0 = time.time()
    ser.write(msg)
    # ser.flush()

    echoed = read_exact(ser, BUF_SIZE, timeout_s=5)
    t1 = time.time()

    if len(echoed) == BUF_SIZE:
        rtt_ms = (t1 - t0) * 1000
        print(f"[Loopback] Received {len(echoed)} bytes, RTT {rtt_ms:.2f} ms")
        print(f"[Loopback] Integrity: {'Passed' if echoed == msg else 'FAILED'}")
    else:
        print(f"[Loopback] Incomplete ({len(echoed)}/{BUF_SIZE})")

    # Clear any leftover bytes before RX-only
    ser.reset_input_buffer()

    NUM_WORDS = 5000
    BYTES_PER_WORD = 4
    TOTAL_BYTES = NUM_WORDS * BYTES_PER_WORD

    WORDS_PER_CHUNK, N_CHUNKS = 5000, 4
    RX_BYTES = WORDS_PER_CHUNK * 4   # 20 kB expected each receive

    # Build 20,000-word buffer [0..19999]
    all_words = list(range(WORDS_PER_CHUNK * N_CHUNKS))
    tx_buf = struct.pack(f"<{len(all_words)}I", *all_words)

    idx = 2
    while True:
        buf = bytearray()
        while len(buf) < TOTAL_BYTES:
            chunk = ser.read(TOTAL_BYTES - len(buf))  # read the remainder
            if not chunk:
                continue  # wait for more data
            buf.extend(chunk)

        # Decode as 10 little-endian uint32_t values
        values = struct.unpack(f"<{NUM_WORDS}I", buf)
        print(f"[{timestamp_ms()}] {values[0]}")

        time.sleep(0.02)
        start = idx * RX_BYTES
        # ser.write(tx_buf[start:start+RX_BYTES]); ser.flush()
        ser.write(msg3)
        # ser.flush()
        idx = (idx + 1) % N_CHUNKS
        print(f"[{timestamp_ms()}] {idx}-th segment sent")



    # while True:
    #     data = ser.read(1)      # read a single byte
    #     if data:
    #         value = data[0]     # convert single byte to int
    #         print(f"[{timestamp_ms()}] {value}")
    #         ser.reset_input_buffer()  # flush any leftover bytes immediately


except serial.SerialException as e:
    print(f"Serial error: {e}")
except KeyboardInterrupt:
    print("\nStopped by user.")
finally:
    if 'ser' in locals() and ser.is_open:
        ser.close()
        print("Serial port closed.")


Connected to COM4 at 12000000 baud.

[Loopback] Received 40000 bytes, RTT 70.78 ms
[Loopback] Integrity: Passed

Stopped by user.
Serial port closed.


In [20]:
import serial
import time
import os
from datetime import datetime
import struct  # add at top

PORT = "COM4"
BAUD = 12000000         # or 12000000 if STM32 is set that way
BUF_SIZE = 40000      # size of random test buffer

import csv, struct

def read_exact(ser, n, timeout_s=5):
    """Read exactly n bytes or return what we got by timeout."""
    end = time.time() + timeout_s
    buf = bytearray()
    while len(buf) < n and time.time() < end:
        chunk = ser.read(n - len(buf))
        if chunk:
            buf += chunk
        else:
            time.sleep(0.001)
    return bytes(buf)

def timestamp_ms():
    """Return current time string with millisecond resolution."""
    return datetime.now().strftime("%H:%M:%S.%f")[:-3]

try:

    WORDS_PER_CHUNK, N_CHUNKS = 5000, 4
    NUM_WORDS2 = WORDS_PER_CHUNK * 4   # 20 kB expected each receiv

    test_values = [i * 256 for i in range(NUM_WORDS2)]

    txBuf = struct.pack(f"<{NUM_WORDS2}I", *test_values)

    test_values2  = []
    for val in range(1, N_CHUNKS + 1):
        test_values2.extend([val] * WORDS_PER_CHUNK)
    txBuf2 = struct.pack(f"<{NUM_WORDS2}I", *test_values2)

    ser = serial.Serial(PORT, BAUD, timeout=0.1)
    print(f"Connected to {PORT} at {BAUD} baud.\n")
    time.sleep(2)

    ser.reset_input_buffer()
    ser.flush()

    # ---- One-time loopback test ----
    msg = os.urandom(BUF_SIZE)
    msg2 = os.urandom(int(BUF_SIZE / 2))
    t0 = time.time()
    ser.write(msg)
    # ser.write(txBuf[0 : 2 * WORDS_PER_CHUNK * BYTES_PER_WORD])
    ser.flush()
    echoed = read_exact(ser, BUF_SIZE, timeout_s=5)
    t1 = time.time()

    if len(echoed) == BUF_SIZE:
        rtt_ms = (t1 - t0) * 1000
        print(f"[Loopback] Received {len(echoed)} bytes, RTT {rtt_ms:.2f} ms")
        print(f"[Loopback] Integrity: {'Passed' if echoed == msg else 'FAILED'}")
    else:
        print(f"[Loopback] Incomplete ({len(echoed)}/{BUF_SIZE})")

    # Clear any leftover bytes before RX-only
    ser.reset_input_buffer()

    NUM_WORDS = 5000
    BYTES_PER_WORD = 4
    TOTAL_BYTES = NUM_WORDS * BYTES_PER_WORD

    idx = 2
    while True:
        buf = bytearray()
        while len(buf) < TOTAL_BYTES:
            chunk = ser.read(TOTAL_BYTES - len(buf))  # read the remainder
            if not chunk:
                continue  # wait for more data
            buf.extend(chunk)

        # Decode as 10 little-endian uint32_t values
        values = struct.unpack(f"<{NUM_WORDS}I", buf)
        print(f"[{timestamp_ms()}] {values[0]}")

        time.sleep(0.01)
        start = idx * WORDS_PER_CHUNK * BYTES_PER_WORD
        # ser.write(tx_buf[start:start + WORDS_PER_CHUNK * BYTES_PER_WORD]); ser.flush()
        ser.write(txBuf2[start : start + WORDS_PER_CHUNK * BYTES_PER_WORD]); ser.flush()
        # ser.write(txBuf[0 : 2 * WORDS_PER_CHUNK * BYTES_PER_WORD])
        # ser.write(txBuf)
        # ser.write(msg2)
        ser.flush()
        idx = (idx + 1) % N_CHUNKS
        print(f"[{timestamp_ms()}] {idx}-th segment sent")



    # while True:
    #     data = ser.read(1)      # read a single byte
    #     if data:
    #         value = data[0]     # convert single byte to int
    #         print(f"[{timestamp_ms()}] {value}")
    #         ser.reset_input_buffer()  # flush any leftover bytes immediately


except serial.SerialException as e:
    print(f"Serial error: {e}")
except KeyboardInterrupt:
    print("\nStopped by user.")
finally:
    if 'ser' in locals() and ser.is_open:
        ser.close()
        print("Serial port closed.")


Connected to COM4 at 12000000 baud.

[Loopback] Received 40000 bytes, RTT 70.67 ms
[Loopback] Integrity: Passed
[13:27:02.560] 10000
[13:27:02.589] 3-th segment sent
[13:27:02.660] 5000
[13:27:02.691] 0-th segment sent
[13:27:02.761] 10000
[13:27:02.790] 1-th segment sent
[13:27:02.860] 5000
[13:27:02.889] 2-th segment sent
[13:27:02.960] 10000
[13:27:02.990] 3-th segment sent
[13:27:03.061] 5000
[13:27:03.090] 0-th segment sent
[13:27:03.160] 10000
[13:27:03.190] 1-th segment sent
[13:27:03.260] 5000
[13:27:03.289] 2-th segment sent
[13:27:03.360] 10000
[13:27:03.390] 3-th segment sent
[13:27:03.460] 5000
[13:27:03.490] 0-th segment sent
[13:27:03.560] 10000
[13:27:03.590] 1-th segment sent
[13:27:03.661] 5000
[13:27:03.689] 2-th segment sent
[13:27:03.761] 10000
[13:27:03.790] 3-th segment sent
[13:27:03.860] 5000
[13:27:03.889] 0-th segment sent
[13:27:03.961] 10000
[13:27:03.989] 1-th segment sent
[13:27:04.057] 5000
[13:27:04.090] 2-th segment sent
[13:27:04.158] 10000
[13:27:04.1

In [21]:
import serial
import time
import os
from datetime import datetime
import struct  # add at top

PORT = "COM4"
BAUD = 12000000         # or 12000000 if STM32 is set that way
BUF_SIZE = 40000      # size of random test buffer

import csv, struct

csv_f = open("capture.csv", "w", newline="")
writer = csv.writer(csv_f)
writer.writerow(["frame", "idx"] + [f"w{c}" for c in range(NUM_WORDS)])
frame = 0

def read_exact(ser, n, timeout_s=5):
    """Read exactly n bytes or return what we got by timeout."""
    end = time.time() + timeout_s
    buf = bytearray()
    while len(buf) < n and time.time() < end:
        chunk = ser.read(n - len(buf))
        if chunk:
            buf += chunk
        else:
            time.sleep(0.001)
    return bytes(buf)

def timestamp_ms():
    """Return current time string with millisecond resolution."""
    return datetime.now().strftime("%H:%M:%S.%f")[:-3]

try:

    WORDS_PER_CHUNK, N_CHUNKS = 5000, 4
    NUM_WORDS2 = WORDS_PER_CHUNK * 4   # 20 kB expected each receiv

    test_values = [i * 256 for i in range(NUM_WORDS2)]

    txBuf = struct.pack(f"<{NUM_WORDS2}I", *test_values)

    test_values2  = []
    for val in range(1, N_CHUNKS + 1):
        test_values2.extend([val] * WORDS_PER_CHUNK)
    txBuf2 = struct.pack(f"<{NUM_WORDS2}I", *test_values2)

    ser = serial.Serial(PORT, BAUD, timeout=0.1)
    print(f"Connected to {PORT} at {BAUD} baud.\n")
    time.sleep(2)

    ser.reset_input_buffer()
    ser.flush()

    # ---- One-time loopback test ----
    msg = os.urandom(BUF_SIZE)
    msg2 = os.urandom(int(BUF_SIZE / 2))
    t0 = time.time()
    ser.write(msg)
    ser.flush()
    echoed = read_exact(ser, BUF_SIZE, timeout_s=5)
    t1 = time.time()

    if len(echoed) == BUF_SIZE:
        rtt_ms = (t1 - t0) * 1000
        print(f"[Loopback] Received {len(echoed)} bytes, RTT {rtt_ms:.2f} ms")
        print(f"[Loopback] Integrity: {'Passed' if echoed == msg else 'FAILED'}")
    else:
        print(f"[Loopback] Incomplete ({len(echoed)}/{BUF_SIZE})")

    # Clear any leftover bytes before RX-only
    ser.reset_input_buffer()

    NUM_WORDS = 5000
    BYTES_PER_WORD = 4
    TOTAL_BYTES = NUM_WORDS * BYTES_PER_WORD

    # idx = 2
    # while True:
    #     buf = bytearray()
    #     while len(buf) < TOTAL_BYTES:
    #         chunk = ser.read(TOTAL_BYTES - len(buf))  # read the remainder
    #         if not chunk:
    #             continue  # wait for more data
    #         buf.extend(chunk)

    #     # Decode as 10 little-endian uint32_t values
    #     values = struct.unpack(f"<{NUM_WORDS}I", buf)
    #     print(f"[{timestamp_ms()}] {values[0]}")

    #     time.sleep(0.01)
    #     start = idx * WORDS_PER_CHUNK * BYTES_PER_WORD
    #     ser.write(txBuf2[start : start + WORDS_PER_CHUNK * BYTES_PER_WORD]); ser.flush()
    #     ser.flush()
    #     idx = (idx + 1) % N_CHUNKS
    #     print(f"[{timestamp_ms()}] {idx}-th segment sent")

    idx = 2
    while True:
        buf = bytearray()
        while len(buf) < TOTAL_BYTES:
            chunk = ser.read(TOTAL_BYTES - len(buf))
            if chunk:
                buf.extend(chunk)

        values = struct.unpack(f"<{NUM_WORDS}I", buf)
        print(f"[{timestamp_ms()}] {values[0]}")

        # store one row per received block
        writer.writerow([frame, idx] + list(values))
        csv_f.flush()
        frame += 1

        time.sleep(0.01)
        start = idx * WORDS_PER_CHUNK * BYTES_PER_WORD
        ser.write(txBuf2[start : start + WORDS_PER_CHUNK * BYTES_PER_WORD]); ser.flush()
        idx = (idx + 1) % N_CHUNKS
        print(f"[{timestamp_ms()}] {idx}-th segment sent")

except serial.SerialException as e:
    print(f"Serial error: {e}")
except KeyboardInterrupt:
    print("\nStopped by user.")
finally:
    if 'ser' in locals() and ser.is_open:
        ser.close()
        print("Serial port closed.")


Connected to COM4 at 12000000 baud.

[Loopback] Received 40000 bytes, RTT 66.94 ms
[Loopback] Integrity: Passed
[13:30:18.542] 10000
[13:30:18.572] 3-th segment sent
[13:30:18.642] 5000
[13:30:18.671] 0-th segment sent
[13:30:18.741] 10000
[13:30:18.771] 1-th segment sent
[13:30:18.842] 5000
[13:30:18.872] 2-th segment sent
[13:30:18.941] 10000
[13:30:18.972] 3-th segment sent
[13:30:19.041] 5000
[13:30:19.072] 0-th segment sent
[13:30:19.137] 10000
[13:30:19.172] 1-th segment sent
[13:30:19.237] 5000
[13:30:19.271] 2-th segment sent
[13:30:19.337] 10000
[13:30:19.371] 3-th segment sent
[13:30:19.437] 5000
[13:30:19.472] 0-th segment sent
[13:30:19.538] 10000
[13:30:19.571] 1-th segment sent
[13:30:19.638] 5000
[13:30:19.671] 2-th segment sent
[13:30:19.738] 10000
[13:30:19.772] 3-th segment sent
[13:30:19.838] 5000
[13:30:19.871] 0-th segment sent
[13:30:19.941] 10000
[13:30:19.971] 1-th segment sent
[13:30:20.042] 5000
[13:30:20.072] 2-th segment sent
[13:30:20.142] 10000
[13:30:20.1